# The Art of Insight


As a place to start, let's investigate the following:

1. What is the average "shelf-life" of a start-up?
2. What do **average investment amounts** look like?
3. Is there a correlation between shelf-life and awarded capital? Between shelf-life and firm?


## Startup Shelf-Life

Let's start by considering shelf-life, and in particular, with a definition:

> The **shelf life** of a startup is its lifetime, defined as the time delta between its **Closed Date** and **Founding Date**. Currently operating startups are taken to have a shelf-life equal to the time delta between today and their founding dates. 


In [1]:
# Best to define/document this here in code—we'll use it below
columns = [
    'Company Name',
    'Founded Date',
    'Closed Date',
    'Status',
    'Headquarters Location',
    'Total Funding Amount',
    'Last Funding Amount'
]

I'm including `Headquarters Location` because I'm curious as to potential correlations between geographical region and shelf-life.

Now that we've defined terms and know what figures we need, we can sketch a plan:

1. Read and reshape `companies.csv` (Pandas)
2. Map **Founded Date** and **Closed Date** Series into [Arrow](http://arrow.readthedocs.io/en/latest/) or `datetime` objects to facilitate timedelta arithmetic.
3. Use **Founded Date** and **Closed Date** to create a **shelf_life** Series.

After generating the `shelf_life` tuples, we can explore the following:

1. Does shelf-life correlate with total funding?
2. Does shelf-life correlate with last funding amount?
3. Is there a relationship between the length of time between last funding date and closing and the last funding amount?

This last one attempts to proxy the question: For firms that close, does the size of their last funding round predict how soon they will close?

In [2]:
# Let's get started...
from os import getcwd
from os.path import join
import pandas as pd

companies = pd.read_csv(join(getcwd(), '../data', 'companies.csv'), encoding='ISO-8859-1')
df = companies[columns]
df.head(20)

,Company Name,Founded Date,Closed Date,Status,Headquarters Location,Total Funding Amount,Last Funding Amount
0,Wokrr,1/1/16,NaN,Closed,"San Jose, California, United States",1111,1111
1,BUILT IMAGES,1/1/12,NaN,Operating,"San Clemente, California, United States",250000,250000
2,Dick or Bro,1/1/12,NaN,Closed,"Los Angeles, California, United States",150000,150000
3,StoryWorth,11/1/12,NaN,Operating,"San Francisco, California, United States",167000,167000
4,Ethonova,1/1/12,NaN,Operating,"San Francisco, California, United States",40000,40000
5,Harbor BioSciences,1/1/12,NaN,Operating,"San Diego, California, United States",2825000,2825000
6,Appsembler,1/1/12,NaN,Operating,"Somerville, Massachusetts, United States",118000,118000
7,Autocosta,4/1/12,NaN,Operating,"Los Angeles, California, United States",1100000,1100000
8,Baboo,1/1/12,NaN,Operating,"Los Angeles, California, United States",250000,250000
9,BindHQ,1/1/12,NaN,Operating,"Tustin, California, United States",500000,500000


Next, we need to turn those **Founded Date** and **Closed Date** columns into something useful, using [Arrow](http://arrow.readthedocs.io/en/latest/).

In [3]:
# Install Arrow, if you haven't already
!pip install arrow

Next, we'll have to convert every string in **Founded Date** into an Arrow object, to facilitate datetime calculations.


In [4]:
import arrow
df['Founded Date'] = df['Founded Date'].apply(lambda date_string: arrow.get(date_string, 'M/D/YY'))

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
df[['Company Name', 'Founded Date']].head(2)

,Company Name,Founded Date
0,Wokrr,2016-01-01T00:00:00+00:00
1,BUILT IMAGES,2012-01-01T00:00:00+00:00


At a glance, less human-readable, but much easier to manipulate.

Let's do the same with the **Closed Date** Series.

In [6]:
from math import isnan

def to_arrow(date_str):
    if type(date_str) is str:
        return arrow.get(date_str, 'M/D/YY')
    elif isnan(date_str):
        return arrow.now()
    
df['Closed Date'] = df['Closed Date'].apply(to_arrow)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
df[['Company Name', 'Closed Date']].head(2)

,Company Name,Closed Date
0,Wokrr,2017-12-21T10:11:57.070826-08:00
1,BUILT IMAGES,2017-12-21T10:11:57.071311-08:00


That yields everything we need to calculate the difference between **Closed Date** and **Founded Date**. Recall that we wanted to include the **Status** for each firm in the reported shelf-life. 


In [8]:
def shelf_life(closed, founded):
    return closed - founded


def reported_shelf_life(shelf_life, status):
    return shelf_life, status

...And then use them:

In [9]:
shelf_lives = [shelf_life(closed, founded) for closed, founded in zip(df['Closed Date'], df['Founded Date'])]
reported = [reported_shelf_life(shelf_life, status) for shelf_life, status in zip(shelf_lives, df['Status'])]

Let's dig in.

In [10]:
### Descriptions
shelf_lives_df = pd.DataFrame(reported, columns=['shelf_life', 'status'])

shelf_lives_df.describe()

,shelf_life
count,8664
mean,1488 days 11:44:17.037300
std,493 days 10:20:18.375298
min,0 days 00:00:00
25%,1085 days 18:11:57.585779
50%,1450 days 18:11:57.725364
75%,1815 days 18:11:57.767609
max,2181 days 18:11:57.817665


In [11]:
# Correlations?
targets = ['Total Funding Amount', 'Last Funding Amount']

for target in targets:
    correlation = df[target].corr(shelf_lives_df['shelf_life'].apply(lambda dtime: dtime.days))
    corr = '%.2f' % correlation
    print(f'The correlation between shelf life and {target} is {corr}.')

The correlation between shelf life and Total Funding Amount is 0.07.
The correlation between shelf life and Last Funding Amount is 0.05.


What's the average shelf-life as a function of operating status?

In [12]:
from collections import Counter

status_counter = Counter(shelf_lives_df)

accumulator = {}
for row in shelf_lives_df.iterrows():
    _, data = row
    shelf_life, status = data
    if accumulator.get(status, 0) == 0:
        accumulator[status] = shelf_life.days
    else:
        accumulator[status] += shelf_life.days
        

for status, shelf_life_days in accumulator.items():
    perc = lambda str: '%.2f' % str
    prop = shelf_life_days / sum(accumulator.values()) * 100
    print(f'Startups with status \'{status.upper()}\' account for {perc(prop)}% of the total lifetime of firms in the data set.')

Startups with status 'CLOSED' account for 2.38% of the total lifetime of firms in the data set.
Startups with status 'OPERATING' account for 90.85% of the total lifetime of firms in the data set.
Startups with status 'WAS ACQUIRED' account for 6.46% of the total lifetime of firms in the data set.
Startups with status 'IPO' account for 0.32% of the total lifetime of firms in the data set.


In [13]:
from helpers import extract, filter_data

usa_data = filter_data(extract())

# Get a list of the cities appearing in our data's list of HQ Locations
ascii_names_of_interest = list(Counter(df['Headquarters Location']).keys())

# Data looks like: 'San Francisco, California, USA'--here, we extract /just/ the city portion ('San Francisc')
cities_of_interest = [ascii_name.split(',')[0] for ascii_name in ascii_names_of_interest]

# Extract only the cities we care about
cities_of_interest_df = usa_data[usa_data['ascii_name'].isin(cities_of_interest)]

usa_data.head(5)

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


KeyError: "['Total Funding Amount'] not in index"

In [ ]:
# ...And then pair with names for convenience, and filter locations with population < 5000
THRESH = 10000

cities_of_interest_df['population'] = cities_of_interest_df['population'] > THRESH

# Remove state/country information from Headquarters Location
companies['ascii_name'] = [location.split(',')[0] for location in companies['Headquarters Location']]

# Next, group by hq_city to sum total funding amounts...
hq_city_funding_totals = companies.groupby('ascii_name')['Total Funding Amount'].sum()
hq_city_funding_totals = hq_city_funding_totals.to_frame().reset_index()

# Add funding total series to our cities_of_interest_df
from numpy import zeros
cities_of_interest_df['funding_total'] = pd.Series(
    zeros(len(cities_of_interest_df)), 
    index=cities_of_interest_df.index
)

In [ ]:
from numpy import zeros

# Add a funding totals column
cities_of_interest_df['funding_total'] = pd.Series(zeros(len(cities_of_interest_df)), index=cities_of_interest_df.index)

print(cities_of_interest_df.columns)

merged = cities_of_interest_df.merge(pd.DataFrame(hq_city_funding_totals), how='inner', left_on='ascii_name', right_on='ascii_name')
merged = merged.drop_duplicates(subset=['ascii_name']).reset_index()
merged

In [ ]:
# Keeping track of some things
avg_total_funding_amount = merged['Total Funding Amount'].mean()
scale = avg_total_funding_amount

scale

    Uf. That's hacky and gross, but it's late, and I don't feel like refactoring.

In [ ]:
! pip install plotly

In [ ]:
import plotly.plotly as py

data = [dict(
    type='scattergeo',
    lat=merged['latitude'],
    lon=merged['longitude'],
    text=merged['ascii_name'],
    markers=dict(
        size=merged['Total Funding Amount'],
        sizemode='area',
        line = dict(width=25, color='rgb(40,40,40)')
    )
)]

layout = dict(
    title='Cities Represented in Startup Data',
    showlegend=False,
    geo=dict(
        scope='usa',
        projection=dict(type='albers usa'),
        showland=True,
        landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor='rgb(255, 255, 255)',
            countrycolor='rgb(255, 255, 255)'
    ),
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False)

This isn't quite satisfactory, but it's a start: At least the trends are _visible_, if not apparent.

Smart next steps would consist of emphasizing these trends. Perhaps by:

1. Creating a histogram to bucket companies based on the range of funding amounts they fall into, and using this to add a color-coded legend
2. Updating the mouseover tooltip to display the funding amount, rather than the city's coordinates and name.

Such cosmetics are left as an exercise to the reader. Have fun.

Some _statistical_ next steps might include:
     
    1. Checking for correlations between funding amounts and latitude/longitude pairs
    2. Normality testing, etc.
    
Due both to the exigencies of time and laziness, these next steps have been ommitted from this write-up. We'll loop back to this closer to **Project 2**.'